In [8]:
import pandas as pd
from preprocess import impute_and_clean

In [9]:
# Función para imputar data
df_ = pd.read_csv("../../data/bronce/hired_employees.csv", index_col=0)
df_

,name,datetime,department_id,job_id
id,,,,
1,Harold Vogt,2021-11-07T02:48:42Z,2.0,96.0
2,Ty Hofer,2021-05-30T05:43:46Z,8.0,NaN
3,Lyman Hadye,2021-09-01T23:27:38Z,5.0,52.0
4,Lotti Crowthe,2021-10-01T13:04:21Z,12.0,71.0
5,Gretna Lording,2021-10-10T22:22:17Z,6.0,80.0
...,...,...,...,...
1995,Goran Yong,2021-08-22T21:57:53Z,2.0,83.0
1996,Cirstoforo Youings,2021-04-01T17:48:42Z,3.0,23.0
1997,Wilek Yurkevich,2021-05-04T12:19:50Z,2.0,132.0


Verificación rápida para saber si antes y después nos encontramos con valores nulos

In [10]:
df_.isnull().sum()

name             19
datetime         14
department_id    21
job_id           16
dtype: int64

In [11]:
cleaned_df = impute_and_clean(df_, nan_threshold=0.5)

In [12]:
cleaned_df.isnull().sum()

name             0
datetime         0
department_id    0
job_id           0
dtype: int64

In [13]:
cleaned_df.dtypes

name              object
datetime          object
department_id    float64
job_id           float64
dtype: object

- Como están en flotantes es mejor castear a enteros, y reemplazar los valores NA por -`1`

In [14]:
cleaned_df

,name,datetime,department_id,job_id
id,,,,
1,Harold Vogt,2021-11-07T02:48:42Z,2.0,96.000000
2,Ty Hofer,2021-05-30T05:43:46Z,8.0,75.431165
3,Lyman Hadye,2021-09-01T23:27:38Z,5.0,52.000000
4,Lotti Crowthe,2021-10-01T13:04:21Z,12.0,71.000000
5,Gretna Lording,2021-10-10T22:22:17Z,6.0,80.000000
...,...,...,...,...
1995,Goran Yong,2021-08-22T21:57:53Z,2.0,83.000000
1996,Cirstoforo Youings,2021-04-01T17:48:42Z,3.0,23.000000
1997,Wilek Yurkevich,2021-05-04T12:19:50Z,2.0,132.000000


In [15]:
cleaned_df['datetime'] = pd.to_datetime(cleaned_df['datetime'], errors='coerce')
cleaned_df['year'] = cleaned_df['datetime'].dt.year
cleaned_df['quarter'] = cleaned_df['datetime'].dt.to_period('Q')
cleaned_df_hired_2021 = cleaned_df[cleaned_df['year'] == 2021]

/tmp/ipykernel_144699/3060871040.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  cleaned_df['quarter'] = cleaned_df['datetime'].dt.to_period('Q')


In [16]:
cleaned_df['department_id'] = cleaned_df['department_id'].fillna(-1).astype(int)
cleaned_df['job_id'] = cleaned_df['job_id'].fillna(-1).astype(int)

# Asegurarnos que la columna 'year' también esté en formato entero
cleaned_df['year'] = cleaned_df['datetime'].dt.year.fillna(-1).astype(int)

In [17]:
cleaned_df_hired_2021.dtypes

name                          object
datetime         datetime64[ns, UTC]
department_id                float64
job_id                       float64
year                         float64
quarter                period[Q-DEC]
dtype: object

In [18]:
cleaned_df.to_csv("../../data/silver/hired_employees_cleaned.csv")

### Preguntas de negocio

1. " Number of employees hired for each job and department in 2021 divided by quarter. The table must be ordered alphabetically by department and job."

2. "  List of ids, name and number of employees hired of each department that hired more employees than the mean of employees hired in 2021 for all the departments, ordered by the number of employees hired (descending). "

In [21]:
jobs_df = pd.read_csv("../../data/bronce/jobs.csv")
jobs_df 

,id,job
0,1,Marketing Assistant
1,2,VP Sales
2,3,Biostatistician IV
3,4,Account Representative II
4,5,VP Marketing
...,...,...
178,179,Software Engineer II
179,180,Statistician IV
180,181,Programmer Analyst I
181,182,Account Representative I


In [22]:
merged_df = pd.merge(cleaned_df, jobs_df, left_on='job_id', right_on='id', suffixes=('', '_job'))

In [23]:
merged_df

,name,datetime,department_id,job_id,year,quarter,id,job
0,Harold Vogt,2021-11-07 02:48:42+00:00,2,96,2021,2021Q4,96,Health Coach III
1,Ty Hofer,2021-05-30 05:43:46+00:00,8,75,2021,2021Q2,75,Community Outreach Specialist
2,Lyman Hadye,2021-09-01 23:27:38+00:00,5,52,2021,2021Q3,52,Structural Analysis Engineer
3,Lotti Crowthe,2021-10-01 13:04:21+00:00,12,71,2021,2021Q4,71,Statistician II
4,Gretna Lording,2021-10-10 22:22:17+00:00,6,80,2021,2021Q4,80,Quality Engineer
...,...,...,...,...,...,...,...,...
1994,Goran Yong,2021-08-22 21:57:53+00:00,2,83,2021,2021Q3,83,Cost Accountant
1995,Cirstoforo Youings,2021-04-01 17:48:42+00:00,3,23,2021,2021Q2,23,Administrative Officer
1996,Wilek Yurkevich,2021-05-04 12:19:50+00:00,2,132,2021,2021Q2,132,Staff Accountant III
1997,Jerry Yven,2021-10-03 14:12:50+00:00,7,100,2021,2021Q4,100,Senior Developer


In [25]:
departments_df = pd.read_csv("../../data/bronce/departments.csv", names = ["id", "department"], index_col=0)
departments_df

,department
id,
1,Product Management
2,Sales
3,Research and Development
4,Business Development
5,Engineering
6,Human Resources
7,Services
8,Support
9,Marketing


In [26]:
merged_df = pd.merge(merged_df, departments_df, left_on='department_id', right_on='id', suffixes=('', '_department'))
merged_df

,name,datetime,department_id,job_id,year,quarter,id,job,department
0,Harold Vogt,2021-11-07 02:48:42+00:00,2,96,2021,2021Q4,96,Health Coach III,Sales
1,Ty Hofer,2021-05-30 05:43:46+00:00,8,75,2021,2021Q2,75,Community Outreach Specialist,Support
2,Lyman Hadye,2021-09-01 23:27:38+00:00,5,52,2021,2021Q3,52,Structural Analysis Engineer,Engineering
3,Lotti Crowthe,2021-10-01 13:04:21+00:00,12,71,2021,2021Q4,71,Statistician II,Accounting
4,Gretna Lording,2021-10-10 22:22:17+00:00,6,80,2021,2021Q4,80,Quality Engineer,Human Resources
...,...,...,...,...,...,...,...,...,...
1994,Goran Yong,2021-08-22 21:57:53+00:00,2,83,2021,2021Q3,83,Cost Accountant,Sales
1995,Cirstoforo Youings,2021-04-01 17:48:42+00:00,3,23,2021,2021Q2,23,Administrative Officer,Research and Development
1996,Wilek Yurkevich,2021-05-04 12:19:50+00:00,2,132,2021,2021Q2,132,Staff Accountant III,Sales
1997,Jerry Yven,2021-10-03 14:12:50+00:00,7,100,2021,2021Q4,100,Senior Developer,Services


In [27]:
# Group by department, job, and quarter and count the number of employees hired
result_df = merged_df.groupby(['department', 'job', 'quarter']).size().reset_index(name='num_employees_hired')
result_df


,department,job,quarter,num_employees_hired
0,Accounting,Account Representative IV,2021Q1,1
1,Accounting,Actuary,2021Q2,1
2,Accounting,Analyst Programmer,2021Q3,1
3,Accounting,Assistant Manager,2022Q1,1
4,Accounting,Associate Professor,2022Q1,1
...,...,...,...,...
1648,Training,Teacher,2021Q2,2
1649,Training,Technical Writer,2021Q3,1
1650,Training,VP Product Management,2021Q1,1
1651,Training,VP Quality Control,2021Q2,1


In [32]:
# Sort the result alphabetically by department and job
sorted_result_df = result_df.sort_values(by=['num_employees_hired'], ascending=False)
sorted_result_df 


,department,job,quarter,num_employees_hired
420,Engineering,Web Designer I,2021Q4,5
358,Engineering,Quality Engineer,2021Q3,5
1452,Support,Programmer IV,2021Q3,4
1506,Support,Systems Administrator IV,2021Q2,4
1519,Support,Web Designer I,2021Q2,4
...,...,...,...,...
607,Human Resources,Staff Accountant IV,2021Q3,1
606,Human Resources,Staff Accountant IV,2021Q2,1
605,Human Resources,Staff Accountant II,2021Q2,1
604,Human Resources,Software Test Engineer IV,2022Q1,1
